In [5]:
import pandas as pd
from datetime import datetime

Create the driver entity dataframe. The entity is defined in `driver_example.py` file

In [6]:
# entity_df generally comes from upstream systems
entity_df = pd.DataFrame.from_dict({
    "driver_id": [1001, 1002, 1003, 1004],
    "event_timestamp": [
        datetime(2021, 4, 12, 10, 59, 42),
        datetime(2021, 4, 12, 8,  12, 10),
        datetime(2021, 4, 12, 16, 40, 26),
        datetime(2021, 4, 12, 15, 1 , 12)
    ]
})

entity_df.head()

,driver_id,event_timestamp
0,1001,2021-04-12 10:59:42
1,1002,2021-04-12 08:12:10
2,1003,2021-04-12 16:40:26
3,1004,2021-04-12 15:01:12


Get historical features from the online feature store relevant to the entity dataframe

In [7]:
from feast import FeatureStore

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
store = FeatureStore(repo_path=".")

training_df = store.get_historical_features(
    entity_df=entity_df, 
    feature_refs = [
        'driver_hourly_stats:conv_rate',
        'driver_hourly_stats:acc_rate',
        'driver_hourly_stats:avg_daily_trips'
    ],
).to_df()

training_df

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,event_timestamp,driver_id,driver_hourly_stats__conv_rate,driver_hourly_stats__acc_rate,driver_hourly_stats__avg_daily_trips
0,2021-04-12 08:12:10+00:00,1002,0.147152,0.289338,222
1,2021-04-12 10:59:42+00:00,1001,0.974248,0.550465,668
2,2021-04-12 15:01:12+00:00,1004,0.618035,0.725218,679
3,2021-04-12 16:40:26+00:00,1003,0.310460,0.161677,342


Fetch a single vector from the online store for inference. The feature table name here is `driver_hourly_stats`
and the feature names are `[conv_rate, acc_rate, avg_daily_trips]`

In [9]:
from pprint import pprint
from feast import FeatureStore

store = FeatureStore(repo_path=".")

feature_vector = store.get_online_features(
    feature_refs=[
        'driver_hourly_stats:conv_rate', # <feature table: feature name>
        'driver_hourly_stats:acc_rate',
        'driver_hourly_stats:avg_daily_trips'
    ],
    entity_rows=[{"driver_id": 1001}]
).to_dict()

pprint(feature_vector)

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'driver_hourly_stats__acc_rate': [0.8544088006019592],
 'driver_hourly_stats__avg_daily_trips': [989],
 'driver_hourly_stats__conv_rate': [0.7719221711158752],
 'driver_id': [1001]}
